In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import yaml

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.four_peaks_generator import FourPeaksGenerator
from mlrose_hiive import SARunner
from mlrose_hiive.fitness.four_peaks import FourPeaks 
from mlrose_hiive import GeomDecay, ExpDecay, ArithDecay
# import mlrose_hiive

In [ ]:
ALGORITHM = 'sa'
MUTATION_RATE_LIST = [0.01, 0.1, 0.5]
TEMPERATURE_LIST = [1, 10, 100]
T_PCT = [0.4]
DECAY_LIST = [ExpDecay, GeomDecay, ArithDecay]
PROBLEM_SIZE_LIST = [20]
ITERATIONS_LIST = [100000]
MAX_ATTEMPTS_LIST = [100000000]
NUM_RUNS = [3]

In [ ]:
problem = FourPeaks(t_pct=0.1)
problem.evaluate([1]*13 + [0]*37)

In [ ]:
max_attempts = 10000000
iterations = 10000
problem_size = 20
t_pct = 0.4

In [2]:
def get_opt_fitness(problem_size):
    R = (1- t_pct)*problem_size
    if R == int(R):
        R = int(R) - 1
    else:
        R = int(R)

    return problem_size + R

OPT_FITNESS = get_opt_fitness(problem_size)
print(OPT_FITNESS)

NameError: name 'problem_size' is not defined

In [ ]:
# SEED = 6
import time
SEED = int(time.time())
problem = FourPeaksGenerator().generate(seed=SEED, size=problem_size, t_pct=t_pct)

runner = SARunner(
    problem=problem,
    experiment_name='SA',
    output_directory='.',
    seed=SEED,
    iteration_list=[iterations],
    max_attempts=max_attempts,
    temperature_list=[5000, ],
    decay_list=[GeomDecay],
)
x, df_run_curves = runner.run()

In [ ]:
df_run_curves['Fitness'].max()

In [ ]:
df_run_curves.tail(10)

In [ ]:
df_path = f'metrics/{ALGORITHM}.csv'
if not os.path.exists(df_path):
    df = pd.DataFrame(columns=[
        'problem_size', 'mutation_rate', 'population_size', 'iterations', 'max_attempts', 
        'run_number', 'best_fitness', 'time', 'fevals'])
    df.to_csv(df_path, index=False)
df = pd.read_csv(df_path)

In [ ]:
all_df = pd.DataFrame()
group_i = 0
run_i = 0
for problem_size in PROBLEM_SIZE_LIST:
    print(f'Problem Size: {problem_size}')
    for iterations in ITERATIONS_LIST:
        print(f'Iterations: {iterations}')
        for max_attempts in MAX_ATTEMPTS_LIST:
            print(f'Max Attempts: {max_attempts}')
            for population_size in POPULATION_SIZES_LIST:
                print(f'Population Size: {population_size}')
                for mutation_rate in MUTATION_RATE_LIST:
                    print(f"Mutation Rate: {mutation_rate}")
                    for i in tqdm(range(NUM_RUNS)):
                        problem = FourPeaksGenerator().generate(seed=SEED+i, size=problem_size, t_pct=T_PCT)
                        sa = GARunner(
                            problem=problem,
                            experiment_name='dontcare',
                            output_directory='./',
                            seed=SEED+i,
                            max_attempts=max_attempts,
                            iteration_list=[iterations],
                            population_sizes=[population_size],
                            mutation_rates=[mutation_rate]
                        )
                        _, df_run_curves = sa.run()
                        df_run_curves['group_number'] = group_i
                        df_run_curves['run_number'] = run_i
                        df_run_curves['problem_size'] = problem_size
                        df_run_curves['max_iterations'] = iterations
                        df_run_curves['max_attempts'] = max_attempts
                        df_run_curves['population_size'] = population_size
                        df_run_curves['mutation_rate'] = mutation_rate

                        print(f"Max Fitness: {df_run_curves['Fitness'].max()}")
                        print(f"Max Iteration: {df_run_curves['Iteration'].max()}")

                        all_df = pd.concat([all_df, df_run_curves], axis=0)
                        run_i += 1
                    group_i += 1
all_df.reset_index(inplace=True, drop=True)

In [ ]:
print(f"Max: {all_df['Fitness'].max()}")
print(f"Min: {all_df['Fitness'].min()}")

In [ ]:
all_df.to_csv(os.path.join(output_dir, 'learning_curve.csv'), index=False)